In [6]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import json

# Cargar el archivo JSON
with open(r"C:\Users\UTIC\Documents\Alejandro\Docs\BOMBS\Archivo .json jtir\prueba.json", 'r', encoding='latin-1') as f:
    data = json.load(f)

# Crear una lista para almacenar las estaciones
estaciones = []

# Recorrer los nodos y estaciones
for nodo, estaciones_list in data.items():
    for estacion in estaciones_list:
        estaciones.append({
            'group': estacion['group'],
            'station_id': estacion['station_id'],
            'name': estacion['name'],
            'lat': estacion['lat'],
            'lon': estacion['lon'],
            'address': estacion['address'],
            'capacity': estacion['capacity']
        })

# Convertir a un DataFrame
df = pd.DataFrame(estaciones)

# Crear la geometría de puntos a partir de latitud y longitud
geometry = [Point(xy) for xy in zip(df['lon'], df['lat'])]
gdf = gpd.GeoDataFrame(df, geometry=geometry)

# Definir el sistema de coordenadas (CRS), generalmente WGS84 (EPSG:4326)
gdf.set_crs(epsg=4326, inplace=True)

# Guardar como archivo .shp
gdf.to_file(r"C:\Users\UTIC\Documents\Alejandro\Docs\BOMBS\Archivo .json jtir\estaciones.shp")


In [7]:
# Agrupar por grupo y formar polígonos convexos (convex hull)
gdf_grouped = gdf.dissolve(by='group')

# Agrupar por grupo y formar polígonos convexos
gdf_grouped['geometry'] = gdf_grouped.convex_hull

# Filtrar geometrías que son Polygon o MultiPolygon
gdf_polygons = gdf_grouped[gdf_grouped.geometry.type.isin(['Polygon', 'MultiPolygon'])]

# Guardar solo los polígonos
gdf_polygons.to_file(r"C:\Users\UTIC\Documents\Alejandro\Docs\BOMBS\Archivo .json jtir\poligonos_estaciones.shp")

# Filtrar geometrías que no son polígonos (e.g. LINESTRING)
gdf_lines = gdf_grouped[gdf_grouped.geometry.type == 'LineString']

# Si quieres guardar las líneas en un archivo separado
if not gdf_lines.empty:
    gdf_lines.to_file(r"C:\Users\UTIC\Documents\Alejandro\Docs\BOMBS\Archivo .json jtir\lineas_estaciones.shp")

